# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.3.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
# https://keras.io/api/preprocessing/
# https://keras.io/api/preprocessing/image/#imagedatagenerator-class

train_datagen = ImageDataGenerator(rescale = 1./255,       # Multiply the data by the value provided
                                                           # (after applying all other transformations).
                                   shear_range = 0.2,      # Shear Intensity (Shear angle in counter-clockwise
                                                           #direction in degrees)
                                   zoom_range = 0.2,       # Range for random zoom
                                   horizontal_flip = True) # Randomly flip inputs horizontally

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64), # 64x64 pixels (150x150 is very long!)
                                                 batch_size = 32,
                                                 class_mode = 'binary') # Binary Outcome

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [3]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
# Apply padding to the image, because every pooling layer is ultimately a filter that will shrink your image
# filters = 32, i.e., number of filters (feature detectors) in the convolution
# Kernel_size = 3, i.e., RGB, images of colour!
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [7]:
# Max polling is used to make computation faster by only taking the
# highest value pixel that mathces the feature that layer is searching for
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
# Flatten to a 1D array of pixels
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
# Adds a full ANN as a layer to detect the features within an image
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [11]:
# Activation fucntion are typically
# - Sigmoid for 0/1 output
# - Relu or softmax for all others
# - tanh and Leaky Relu also possible but need to be tested for performance
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
# This Loss Function is meant for binary classification (categorical_crossentropy for multi class)
# Adam optimizer uses stochastic gradient descent, i.e., weights are updated after every row (picture)
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25) # 25 times through the training dataset

Epoch 1/25
250/250 [==============================] - 45s 179ms/step - loss: 0.6768 - accuracy: 0.5872 - val_loss: 0.6396 - val_accuracy: 0.6190
Epoch 2/25
250/250 [==============================] - 47s 188ms/step - loss: 0.6333 - accuracy: 0.6480 - val_loss: 0.5974 - val_accuracy: 0.7000
Epoch 3/25
250/250 [==============================] - 39s 158ms/step - loss: 0.6021 - accuracy: 0.6764 - val_loss: 0.6248 - val_accuracy: 0.6420
Epoch 4/25
250/250 [==============================] - 40s 158ms/step - loss: 0.5624 - accuracy: 0.7064 - val_loss: 0.6455 - val_accuracy: 0.6160
Epoch 5/25
250/250 [==============================] - 39s 157ms/step - loss: 0.5338 - accuracy: 0.7296 - val_loss: 0.5784 - val_accuracy: 0.6860
Epoch 6/25
250/250 [==============================] - 42s 167ms/step - loss: 0.5101 - accuracy: 0.7442 - val_loss: 0.5025 - val_accuracy: 0.7625
Epoch 7/25
250/250 [==============================] - 43s 173ms/step - loss: 0.4926 - accuracy: 0.7600 - val_loss: 0.5084 - val_ac

## Part 4 - Making a single prediction

In [14]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [15]:
print(prediction)

dog


## Visualize your CNN

In [1]:
#!pip install pydot
#!pip install graphviz
#!pip install pydotplus

from tensorflow.keras.utils import plot_model

In [2]:
tf.keras.utils.plot_model(cnn, to_file='model.png', show_shapes=True, show_layer_names=True)

NameError: name 'tf' is not defined